In [5]:
import numpy as np
import cv2
from PIL import Image

from Algorithm import permute_rows, repermute_columns, repermute_rows, permute_columns
from Image import blockshaped
from Metrics import Entr, Hist, EQ

In [6]:
# constants
r = 3.73421
key = 3.24725
lkey = key - int(key)

In [8]:
img_path = 'station.jpg'
img = cv2.imread(img_path, 0)

n = len(img)
img_pi = np.zeros((n, n), dtype=np.uint8)
img_spi = np.zeros((n, n), dtype=np.uint8)

In [9]:
# step 3
S_a = np.zeros(n, dtype=float)
S_b = np.zeros(n, dtype=float)

S_a[0] = lkey
S_b[0] = 1 - lkey

for i in range(1, n):
    S_a[i] =  r*S_a[i-1]*(1 - S_a[i-1])    
    S_b[i] =  r*S_b[i-1]*(1 - S_b[i-1])

S_1n = np.concatenate((S_a[0:n//2], S_b[n//2:n]))
S_2n = np.concatenate((S_b[0:n//2], S_a[n//2:n]))
S_3n = np.concatenate((S_a[0:n//2], S_b[0:n//2]))
S_4n = np.concatenate((S_b[n//2:n], S_a[n//2:n]))

img_pi = permute_rows(img, S_1n, n)
img_pi = permute_rows(img_pi, S_2n, n)
img_pi = permute_rows(img_pi, S_3n, n)
img_pi = permute_rows(img_pi, S_4n, n)

img_spi = permute_columns(img_pi, S_1n, n)
img_spi = permute_columns(img_spi, S_2n, n)
img_spi = permute_columns(img_spi, S_3n, n)
img_spi = permute_columns(img_spi, S_4n, n)


# step 4
img_fpi = permute_rows(img_spi, S_a, n)
img_fpi = permute_columns(img_fpi, S_b, n)


# step 5
L_1 = np.zeros((n**2)//2, dtype=float)
L_2 = np.zeros((n**2)//2, dtype=float)
LN_1 = np.zeros((n**2)//2, dtype=int)
LN_2 = np.zeros((n**2)//2, dtype=int)

L_1[0] = (1 - lkey)/3
L_2[0] = (1 - lkey)/4

for i in range(1, (n**2)//2):
    L_1[i] =  r*L_1[i-1]*(1 - L_1[i-1])    
    L_2[i] =  r*L_2[i-1]*(1 - L_2[i-1])
    
for i in range((n**2)//2):
    LN_1[i] = (L_1[i] * (10**14)) % 256   
    LN_2[i] = (L_2[i] * (10**14)) % 256
    
# step 6
img_fpi_str = img_fpi.flatten()

C = np.zeros(n**2, dtype=np.uint8)
C[0] = LN_1[0] ^ LN_2[0] ^ img_fpi_str[0]
C[1] = LN_2[0] ^ C[0] ^ img_fpi_str[1]

for j in range(1, (n**2)//2):
    C[2*j] = LN_1[j] ^ C[2*j - 1] ^ img_fpi_str[2*j]
    C[2*j + 1] = LN_2[j] ^ C[2*j] ^ img_fpi_str[2*j + 1]

In [10]:
# save encrypted image
arr = np.reshape(C, (-1, n))

img_en = Image.fromarray(arr, 'L')
img_en.save("encrypted.jpg")

In [11]:
# decryption

# step 2
D_1 = np.zeros((n**2)//2, dtype=float)
D_2 = np.zeros((n**2)//2, dtype=float)
DN_1 = np.zeros((n**2)//2, dtype=int)
DN_2 = np.zeros((n**2)//2, dtype=int)

D_1[0] = (1 - lkey)/3
D_2[0] = (1 - lkey)/4

for i in range(1, (n**2)//2):
    D_1[i] =  r*D_1[i-1]*(1 - D_1[i-1])    
    D_2[i] =  r*D_2[i-1]*(1 - D_2[i-1])
    
for i in range((n**2)//2):
    DN_1[i] = (D_1[i] * (10**14)) % 256   
    DN_2[i] = (D_2[i] * (10**14)) % 256

# step 3
img_di = np.zeros(n**2, dtype=int)

DI = np.zeros(n**2, dtype=np.uint8)
DI[0] = DN_1[0] ^ DN_2[0] ^ C[0]
DI[1] = DN_2[0] ^ C[0] ^ C[1]

for j in range(1, (n**2)//2):
    DI[2*j] = DN_1[j] ^ C[2*j - 1] ^ C[2*j]
    DI[2*j + 1] = DN_2[j] ^ C[2*j] ^ C[2*j + 1]
    
arr = np.reshape(DI, (-1, n))

In [12]:
# step 4
img_1 = repermute_columns(arr, S_b, n)
img_1 = repermute_rows(img_1, S_a, n)

# step 5
img_2 = repermute_columns(img_1, S_4n, n)
img_2 = repermute_columns(img_2, S_3n, n)
img_2 = repermute_columns(img_2, S_2n, n)
img_2 = repermute_columns(img_2, S_1n, n)

img_3 = repermute_rows(img_2, S_4n, n)
img_3 = repermute_rows(img_3, S_3n, n)
img_3 = repermute_rows(img_3, S_2n, n)
img_3 = repermute_rows(img_3, S_1n, n)

In [13]:
# save decrypted image

im_1 = Image.fromarray(img_3, 'L')
im_1.save("decrypted.jpg")